<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Read the data**

In [2]:
# PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')
PATH_TO_DATA = Path('./data/')

In [3]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')

In [4]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
Month            100000 non-null object
DayofMonth       100000 non-null object
DayOfWeek        100000 non-null object
DepTime          100000 non-null int64
UniqueCarrier    100000 non-null object
Origin           100000 non-null object
Dest             100000 non-null object
Distance         100000 non-null int64
dtypes: int64(2), object(6)
memory usage: 6.1+ MB


**Create only one feature - “flight” (this you need to improve - add more features)**

In [7]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

train_df['CarrierOrigin'] = train_df['UniqueCarrier'] + ': ' + train_df['Origin']
test_df['CarrierOrigin'] = test_df['UniqueCarrier'] + ': ' + test_df['Origin']

train_df['CarrierDest'] = train_df['UniqueCarrier'] + ': ' + train_df['Dest']
test_df['CarrierDest'] = test_df['UniqueCarrier'] + ': ' + test_df['Dest']

train_df['CarrierRout'] = train_df['UniqueCarrier'] + ': ' + train_df['flight']
test_df['CarrierRout'] = test_df['UniqueCarrier'] + ': ' + test_df['flight']

train_df['Dep_hour'] = train_df['DepTime']//100
train_df['Dep_hour'].replace(to_replace=[24,25], value=0, inplace=True)
train_df['Dep_hour'] = train_df['Dep_hour'].astype('object')
test_df['Dep_hour'] = test_df['DepTime']//100
test_df['Dep_hour'].replace(to_replace=[24,25], value=0, inplace=True)
test_df['Dep_hour'] = test_df['Dep_hour'].astype('object')

train_df['Dep_minute'] = train_df['DepTime']%100
train_df['Dep_minute'] = train_df['Dep_minute']
test_df['Dep_minute'] = test_df['DepTime']%100
test_df['Dep_minute'] = test_df['Dep_minute']

train_df['CarrierOriginDepHour'] = train_df['CarrierOrigin'] + ': ' + train_df['Dep_hour'].astype('str')
test_df['CarrierOriginDepHour'] = test_df['CarrierOrigin'] + ': ' + test_df['Dep_hour'].astype('str')

# train_df['CarrierDistanceDepHour'] = train_df['UniqueCarrier'] + ': ' + train_df['Distance'].astype('str') + ': ' + train_df['Dep_hour'].astype('str')
# test_df['CarrierDistanceDepHour'] = test_df['UniqueCarrier'] + ': ' + test_df['Distance'].astype('str') + ': ' + test_df['Dep_hour'].astype('str')

train_df['CarrierRoutDepHour'] = train_df['CarrierRout'] + ': ' + train_df['Dep_hour'].astype('str')
test_df['CarrierRoutDepHour'] = test_df['CarrierRout'] + ': ' + test_df['Dep_hour'].astype('str')

train_df = train_df.drop(['CarrierOrigin'], axis=1)
test_df = test_df.drop(['CarrierOrigin'], axis=1)
# Season category

def seasons(month):
    if month in ['12', '1', '2']:
        return 'winter'
    elif month in ['3', '4', '5']:
        return 'spring'
    elif month in ['6', '7', '8']:
        return 'summer'
    elif month in ['9', '10', '11']:
        return 'autumn'

train_df['Season'] = train_df['Month'].apply(lambda x: x.split('-')[1]).apply(seasons)
test_df['Season'] = test_df['Month'].apply(lambda x: x.split('-')[1]).apply(seasons)

# train_df['Dep_hour_flag'] = ((train_df['Dep_hour'] >= 6) & (train_df['Dep_hour'] < 23)).astype('int').astype('object')
# test_df['Dep_hour_flag'] = ((test_df['Dep_hour'] >= 6) & (test_df['Dep_hour'] < 23)).astype('int').astype('object')

# # Carrier Quality

# total_flights_by_carrier = train_df.groupby(['UniqueCarrier']).count().sort_values(by='dep_delayed_15min', ascending=False)['dep_delayed_15min']

# number_of_delays_by_carrier = train_df[train_df['dep_delayed_15min'] == 'Y'].groupby(['UniqueCarrier']).count().sort_values(by='dep_delayed_15min', ascending=False)['dep_delayed_15min']

# prcnt_of_delays = ((number_of_delays_by_carrier*100)/total_flights_by_carrier).sort_values(ascending=True)
# prcnt_of_delays = prcnt_of_delays.reset_index().rename({'dep_delayed_15min':'% of delayed flights'}, axis=1)

# list_of_good_carriers = list(
#     prcnt_of_delays[
#         prcnt_of_delays['% of delayed flights'] < prcnt_of_delays['% of delayed flights'].median()]['UniqueCarrier'])

# # list_of_bad_carriers = list(
# #     prcnt_of_delays[
# #         prcnt_of_delays['% of delayed flights'] > prcnt_of_delays['% of delayed flights'].median()]['UniqueCarrier'])

# def carrier_quality(carrier):    
#     if carrier in list_of_good_carriers:
#         return 'Good'
#     # elif carrier in list_of_bad_carriers:
#     #     return 'Bad'
#     else:
#         return 'Average'

# train_df['CarrierQuality'] = train_df['UniqueCarrier'].apply(carrier_quality)
# test_df['CarrierQuality'] = test_df['UniqueCarrier'].apply(carrier_quality)



# # Weekend

# def weekend(day):
#     if day in ['5', '6', '7']:
#         return 1
#     else:
#         return 0

# train_df['Weekend'] = train_df['DayOfWeek'].apply(lambda x: x.split('-')[1]).apply(weekend)#.astype('object')
# test_df['Weekend'] = test_df['DayOfWeek'].apply(lambda x: x.split('-')[1]).apply(weekend)#.astype('object')


# # Seasons binary

# train_df['Winter'] = train_df['Month'].apply(lambda x: x.split('-')[1]).isin(['12', '1', '2']).astype(int)#.astype('object')
# test_df['Winter'] = test_df['Month'].apply(lambda x: x.split('-')[1]).isin(['12', '1', '2']).astype(int)#.astype('object')

# train_df['Spring'] = train_df['Month'].apply(lambda x: x.split('-')[1]).isin(['3', '4', '5']).astype(int)#.astype('object')
# test_df['Spring'] = test_df['Month'].apply(lambda x: x.split('-')[1]).isin(['3', '4', '5']).astype(int)#.astype('object')

# train_df['Summer'] = train_df['Month'].apply(lambda x: x.split('-')[1]).isin(['6', '7', '8']).astype(int)#.astype('object')
# test_df['Summer'] = test_df['Month'].apply(lambda x: x.split('-')[1]).isin(['6', '7', '8']).astype(int)#.astype('object')

# train_df['Autumn'] = train_df['Month'].apply(lambda x: x.split('-')[1]).isin(['9', '10', '11']).astype(int)#.astype('object')
# test_df['Autumn'] = test_df['Month'].apply(lambda x: x.split('-')[1]).isin(['9', '10', '11']).astype(int)#.astype('object')



# def part_of_day(hour):
#     if int(hour) in range(6,12,1):
#         return 'morning'
#     if int(hour) in range(12,18,1):
#         return 'day'
#     if int(hour) in range(18, 24):
#         return 'evening'
#     else:
#         return 'night'
# 
# train_df['PartOfTheDay'] = train_df['Dep_hour'].apply(part_of_day).astype('object')
# test_df['PartOfTheDay'] = test_df['Dep_hour'].apply(part_of_day).astype('object')
# 

# def distance_cat(distance):
#     if int(distance) < 500:
#         return 'near'
#     if 500 <= int(distance) < 1000:
#         return 'middle'
#     if int(distance) >= 1000:
#         return 'far'
#     
# # train_df['Distance'].mean()    
# test_df['DistCat'] = test_df['Distance'].astype('object')#.apply(distance_cat)
# train_df['DistCat'] = train_df['Distance'].astype('object')#.apply(distance_cat)

# train_df['DepTime'] = train_df['DepTime'].astype('object')
# test_df['DepTime'] = test_df['DepTime'].astype('object')

test_df.head(3)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,flight,CarrierDest,CarrierRout,Dep_hour,Dep_minute,CarrierOriginDepHour,CarrierRoutDepHour,Season
0,c-7,c-25,c-3,615,YV,MRY,PHX,598,MRY-->PHX,YV: PHX,YV: MRY-->PHX,6,15,YV: MRY: 6,YV: MRY-->PHX: 6,summer
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235,LAS-->HOU,WN: HOU,WN: LAS-->HOU,7,39,WN: LAS: 7,WN: LAS-->HOU: 7,spring
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577,GSP-->ORD,MQ: ORD,MQ: GSP-->ORD,6,51,MQ: GSP: 6,MQ: GSP-->ORD: 6,winter


In [8]:
# train_df.groupby(['CarrierRout', 'DayOfWeek'])[['flight']].count()

In [9]:
test_df.isna().sum()

Month                   0
DayofMonth              0
DayOfWeek               0
DepTime                 0
UniqueCarrier           0
Origin                  0
Dest                    0
Distance                0
flight                  0
CarrierDest             0
CarrierRout             0
Dep_hour                0
Dep_minute              0
CarrierOriginDepHour    0
CarrierRoutDepHour      0
Season                  0
dtype: int64

In [10]:
train_df.drop('dep_delayed_15min', axis=1).columns == test_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [11]:
train_df.shape[0] == test_df.shape[0]

True

In [12]:
train_df.dtypes

Month                   object
DayofMonth              object
DayOfWeek               object
DepTime                  int64
UniqueCarrier           object
Origin                  object
Dest                    object
Distance                 int64
dep_delayed_15min       object
flight                  object
CarrierDest             object
CarrierRout             object
Dep_hour                object
Dep_minute               int64
CarrierOriginDepHour    object
CarrierRoutDepHour      object
Season                  object
dtype: object

**Remember indexes of categorical features (to be passed to CatBoost)**

In [13]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([ 0,  1,  2,  4,  5,  6,  8,  9, 10, 11, 13, 14, 15], dtype=int64)

In [14]:
train_df.drop('dep_delayed_15min', axis=1).columns[categ_feat_idx]

Index(['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest',
       'flight', 'CarrierDest', 'CarrierRout', 'Dep_hour',
       'CarrierOriginDepHour', 'CarrierRoutDepHour', 'Season'],
      dtype='object')

**Allocate a hold-out set (a.k.a. a validation set) to validate the model**

In [15]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [16]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

**Train Catboost with default arguments, passing only the indexes of categorical features.**

In [17]:
ctb = CatBoostClassifier(random_seed=17,
                         silent=True,
                         task_type="GPU",
                         devices="0",
                         border_count=254,
                         iterations=1500)

In [18]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx);

Wall time: 8min 23s


In [19]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

**We got some 0.756 ROC AUC on the hold-out set.**

In [20]:
roc_auc_score(y_valid, ctb_valid_pred)

0.823928741052146

In [21]:
df = pd.DataFrame({'feature_name': train_df.drop('dep_delayed_15min', axis=1).columns,
                   'importance': ctb.feature_importances_})

df.sort_values(by='importance', ascending=False)

,feature_name,importance
12,Dep_minute,19.079790
11,Dep_hour,12.724715
14,CarrierRoutDepHour,12.060207
3,DepTime,7.930531
13,CarrierOriginDepHour,6.625573
10,CarrierRout,5.152296
4,UniqueCarrier,4.850050
7,Distance,4.333131
5,Origin,4.043542
0,Month,3.886226


**Train on the whole train set, make prediction on the test set. We got ~0.734 in the competition - "Catboost starter" baseline**

In [22]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

Wall time: 8min 40s


In [23]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [24]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('ctb_pred.csv')

In [25]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 1 columns):
dep_delayed_15min    100000 non-null float64
dtypes: float64(1)
memory usage: 1.5 MB


Now's your turn! Go and improve the model to beat **"A2 baseline (10 credits)"** - **0.75914** LB score. It's crucial to come up with some good features. 

For discussions, stick to the **#a2_kaggle_fall2019** thread in the **mlcourse_ai_news** [ODS Slack](http://opendatascience.slack.com) channel. Serhii Romanenko (@serhii_romanenko) will be there to help. 

Welcome to Kaggle!

<img src='https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg' width=50%>
*from the ["Nerd Laughing Loud"](https://www.kaggle.com/general/76963) thread.*